In [1]:
from theano.sandbox import cuda
cuda.use('cpu')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


## Setup

In [3]:
batch_size=64

In [4]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
X_test = np.expand_dims(X_test,1)
X_train = np.expand_dims(X_train,1)

In [6]:
X_train.shape

(60000, 1, 28, 28)

In [7]:
y_train.shape

(60000,)

In [8]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [9]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [10]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [11]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [12]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [13]:
def norm_input(x): return (x-mean_px)/std_px

## Linear model

In [14]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [15]:
lm = get_lin_model()

/home/farlion/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [16]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [17]:
batches

In [18]:
lm.fit_generator(batches, batches.batch_size, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/1
64/64 [==============================] - 0s - loss: 2.6166 - acc: 0.1406 - val_loss: 2.4392 - val_acc: 0.2500


In [19]:
lm.optimizer.lr=0.1

In [21]:
lm.fit_generator(batches, batches.batch_size, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/1
64/64 [==============================] - 0s - loss: 2.6284 - acc: 0.1875 - val_loss: 2.2685 - val_acc: 0.2812


In [22]:
lm.optimizer.lr=0.01

In [23]:
lm.fit_generator(batches, batches.batch_size, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/4
64/64 [==============================] - 0s - loss: 2.2611 - acc: 0.1719 - val_loss: 2.2491 - val_acc: 0.2812
Epoch 2/4
64/64 [==============================] - 0s - loss: 2.4246 - acc: 0.2500 - val_loss: 2.1031 - val_acc: 0.2656
Epoch 3/4
64/64 [==============================] - 0s - loss: 1.9307 - acc: 0.3438 - val_loss: 1.8618 - val_acc: 0.3281
Epoch 4/4
64/64 [==============================] - 0s - loss: 1.7970 - acc: 0.3750 - val_loss: 1.6259 - val_acc: 0.4844


## Single dense layer

In [24]:
def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(512, activation='softmax'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [25]:
fc = get_fc_model()

/home/farlion/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [26]:
fc.fit_generator(batches, batches.batch_size, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/1
64/64 [==============================] - 0s - loss: 2.3014 - acc: 0.1094 - val_loss: 2.2992 - val_acc: 0.2812


In [27]:
fc.optimizer.lr=0.1

In [28]:
fc.fit_generator(batches, batches.batch_size, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/4
64/64 [==============================] - 0s - loss: 2.2993 - acc: 0.2188 - val_loss: 2.2938 - val_acc: 0.4062
Epoch 2/4
64/64 [==============================] - 0s - loss: 2.2965 - acc: 0.4062 - val_loss: 2.2948 - val_acc: 0.3125
Epoch 3/4
64/64 [==============================] - 0s - loss: 2.2917 - acc: 0.4062 - val_loss: 2.2896 - val_acc: 0.4219
Epoch 4/4
64/64 [==============================] - 0s - loss: 2.2913 - acc: 0.4531 - val_loss: 2.2828 - val_acc: 0.5781


In [29]:
fc.optimizer.lr=0.01

In [30]:
fc.fit_generator(batches, batches.batch_size, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/4
64/64 [==============================] - 0s - loss: 2.2854 - acc: 0.5312 - val_loss: 2.2864 - val_acc: 0.4062
Epoch 2/4
64/64 [==============================] - 0s - loss: 2.2825 - acc: 0.6250 - val_loss: 2.2796 - val_acc: 0.5781
Epoch 3/4
64/64 [==============================] - 0s - loss: 2.2789 - acc: 0.6562 - val_loss: 2.2758 - val_acc: 0.5938
Epoch 4/4
64/64 [==============================] - 0s - loss: 2.2767 - acc: 0.5156 - val_loss: 2.2688 - val_acc: 0.6875


## Basic 'VGG-style' CNN

In [31]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [32]:
model = get_model()

/home/farlion/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [36]:
model.fit_generator(batches, batches.batch_size, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/1
64/64 [==============================] - 0s - loss: 2.3912 - acc: 0.0938 - val_loss: 2.1568 - val_acc: 0.0625


In [37]:
model.optimizer.lr=0.1

In [38]:
model.fit_generator(batches, batches.batch_size, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/1
64/64 [==============================] - 0s - loss: 2.1503 - acc: 0.0625 - val_loss: 2.0527 - val_acc: 0.4375


In [39]:
model.optimizer.lr=0.01

In [40]:
model.fit_generator(batches, batches.batch_size, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/8
64/64 [==============================] - 0s - loss: 2.0724 - acc: 0.4844 - val_loss: 1.9557 - val_acc: 0.5625
Epoch 2/8
64/64 [==============================] - 0s - loss: 1.9639 - acc: 0.5625 - val_loss: 1.8232 - val_acc: 0.6719
Epoch 3/8
64/64 [==============================] - 0s - loss: 1.8576 - acc: 0.5625 - val_loss: 1.6029 - val_acc: 0.6562
Epoch 4/8
64/64 [==============================] - 0s - loss: 1.7152 - acc: 0.5469 - val_loss: 1.3334 - val_acc: 0.7344
Epoch 5/8
64/64 [==============================] - 0s - loss: 1.4250 - acc: 0.6562 - val_loss: 1.2745 - val_acc: 0.5938
Epoch 6/8
64/64 [==============================] - 0s - loss: 1.1268 - acc: 0.7500 - val_loss: 0.8266 - val_acc: 0.7344
Epoch 7/8
64/64 [==============================] - 0s - loss: 0.9042 - acc: 0.7500 - val_loss: 0.7526 - val_acc: 0.7031
Epoch 8/8
64/64 [==============================] - 0s - loss: 0.6594 - acc: 0.7969 - val_loss: 0.7550 - val_acc: 0.8125


## Data augmentation

In [41]:
model = get_model()

/home/farlion/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_4 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [42]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [43]:
model.fit_generator(batches, batches.batch_size, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/1
64/64 [==============================] - 0s - loss: 2.2844 - acc: 0.1719 - val_loss: 2.2706 - val_acc: 0.1250


In [44]:
model.optimizer.lr=0.1

In [45]:
model.fit_generator(batches, batches.batch_size, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/4
64/64 [==============================] - 0s - loss: 2.3890 - acc: 0.0625 - val_loss: 2.2465 - val_acc: 0.1250
Epoch 2/4
64/64 [==============================] - 0s - loss: 2.2220 - acc: 0.1875 - val_loss: 2.2297 - val_acc: 0.2188
Epoch 3/4
64/64 [==============================] - 0s - loss: 2.2266 - acc: 0.1875 - val_loss: 2.2206 - val_acc: 0.3438
Epoch 4/4
64/64 [==============================] - 0s - loss: 2.2224 - acc: 0.2656 - val_loss: 2.0846 - val_acc: 0.2656


In [46]:
model.optimizer.lr=0.01

In [47]:
model.fit_generator(batches, batches.batch_size, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/8
64/64 [==============================] - 0s - loss: 2.0885 - acc: 0.3906 - val_loss: 2.1107 - val_acc: 0.2969
Epoch 2/8
64/64 [==============================] - 0s - loss: 2.0717 - acc: 0.2812 - val_loss: 2.0208 - val_acc: 0.3750
Epoch 3/8
64/64 [==============================] - 0s - loss: 2.0189 - acc: 0.3438 - val_loss: 1.8689 - val_acc: 0.3750
Epoch 4/8
64/64 [==============================] - 0s - loss: 1.9620 - acc: 0.3750 - val_loss: 1.8290 - val_acc: 0.4531
Epoch 5/8
64/64 [==============================] - 0s - loss: 1.7699 - acc: 0.4531 - val_loss: 1.4932 - val_acc: 0.5938
Epoch 6/8
64/64 [==============================] - 0s - loss: 1.5090 - acc: 0.5781 - val_loss: 1.2357 - val_acc: 0.6875
Epoch 7/8
64/64 [==============================] - 0s - loss: 1.5193 - acc: 0.4844 - val_loss: 1.0790 - val_acc: 0.7031
Epoch 8/8
64/64 [==============================] - 0s - loss: 1.1831 - acc: 0.6406 - val_loss: 1.1941 - val_acc: 0.6250


In [48]:
model.optimizer.lr=0.001

In [49]:
model.fit_generator(batches, batches.batch_size, nb_epoch=14, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/14
64/64 [==============================] - 0s - loss: 1.2289 - acc: 0.6719 - val_loss: 1.3109 - val_acc: 0.5469
Epoch 2/14
64/64 [==============================] - 0s - loss: 1.3426 - acc: 0.5156 - val_loss: 1.1596 - val_acc: 0.6250
Epoch 3/14
64/64 [==============================] - 0s - loss: 1.1644 - acc: 0.5938 - val_loss: 1.2596 - val_acc: 0.6250
Epoch 4/14
64/64 [==============================] - 0s - loss: 1.1526 - acc: 0.6094 - val_loss: 0.9436 - val_acc: 0.7031
Epoch 5/14
64/64 [==============================] - 0s - loss: 0.8659 - acc: 0.6406 - val_loss: 1.3352 - val_acc: 0.5000
Epoch 6/14
64/64 [==============================] - 0s - loss: 1.2300 - acc: 0.6094 - val_loss: 1.0783 - val_acc: 0.6562
Epoch 7/14
64/64 [==============================] - 0s - loss: 1.0158 - acc: 0.7188 - val_loss: 1.0174 - val_acc: 0.6875
Epoch 8/14
64/64 [==============================] - 0s - loss: 0.7928 - acc: 0.7188 - val_loss: 0.6207 - val_acc: 0.7969
Epoch 9/14
64/64 [==============

In [50]:
model.optimizer.lr=0.0001

In [51]:
model.fit_generator(batches, batches.batch_size, nb_epoch=10, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/10
64/64 [==============================] - 0s - loss: 0.5952 - acc: 0.8281 - val_loss: 0.7342 - val_acc: 0.7656
Epoch 2/10
64/64 [==============================] - 0s - loss: 0.6981 - acc: 0.8438 - val_loss: 0.8610 - val_acc: 0.7344
Epoch 3/10
64/64 [==============================] - 0s - loss: 0.6482 - acc: 0.7344 - val_loss: 0.4779 - val_acc: 0.8594
Epoch 4/10
64/64 [==============================] - 0s - loss: 0.7330 - acc: 0.7812 - val_loss: 0.5778 - val_acc: 0.7812
Epoch 5/10
64/64 [==============================] - 0s - loss: 1.1774 - acc: 0.6562 - val_loss: 0.5890 - val_acc: 0.7969
Epoch 6/10
64/64 [==============================] - 0s - loss: 0.9252 - acc: 0.7812 - val_loss: 0.5086 - val_acc: 0.8438
Epoch 7/10
64/64 [==============================] - 0s - loss: 0.5169 - acc: 0.7812 - val_loss: 0.8148 - val_acc: 0.7812
Epoch 8/10
64/64 [==============================] - 0s - loss: 0.8732 - acc: 0.7969 - val_loss: 0.4501 - val_acc: 0.8906
Epoch 9/10
64/64 [==============

## Batchnorm + data augmentation

In [52]:
def get_model_bn():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [53]:
model = get_model_bn()

/home/farlion/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [54]:
model.fit_generator(batches, batches.batch_size, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/1
64/64 [==============================] - 0s - loss: 3.0337 - acc: 0.1562 - val_loss: 2.2785 - val_acc: 0.1719


In [55]:
model.optimizer.lr=0.1

In [56]:
model.fit_generator(batches, batches.batch_size, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/4
64/64 [==============================] - 0s - loss: 2.1923 - acc: 0.2500 - val_loss: 2.2678 - val_acc: 0.2031
Epoch 2/4
64/64 [==============================] - 0s - loss: 1.4511 - acc: 0.5156 - val_loss: 2.2615 - val_acc: 0.1719
Epoch 3/4
64/64 [==============================] - 0s - loss: 1.3782 - acc: 0.5625 - val_loss: 2.2125 - val_acc: 0.4219
Epoch 4/4
64/64 [==============================] - 0s - loss: 1.2865 - acc: 0.5625 - val_loss: 2.1974 - val_acc: 0.4844


In [57]:
model.optimizer.lr=0.01

In [58]:
model.fit_generator(batches, batches.batch_size, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/12
64/64 [==============================] - 0s - loss: 1.5898 - acc: 0.5938 - val_loss: 2.1826 - val_acc: 0.5469
Epoch 2/12
64/64 [==============================] - 0s - loss: 1.0248 - acc: 0.7188 - val_loss: 2.1596 - val_acc: 0.5469
Epoch 3/12
64/64 [==============================] - 0s - loss: 0.8667 - acc: 0.7188 - val_loss: 2.1394 - val_acc: 0.5938
Epoch 4/12
64/64 [==============================] - 0s - loss: 1.1608 - acc: 0.6719 - val_loss: 2.1286 - val_acc: 0.5469
Epoch 5/12
64/64 [==============================] - 0s - loss: 1.0471 - acc: 0.5938 - val_loss: 2.1138 - val_acc: 0.5000
Epoch 6/12
64/64 [==============================] - 0s - loss: 0.9476 - acc: 0.7031 - val_loss: 2.1135 - val_acc: 0.4375
Epoch 7/12
64/64 [==============================] - 0s - loss: 0.6269 - acc: 0.8281 - val_loss: 2.1604 - val_acc: 0.3281
Epoch 8/12
64/64 [==============================] - 0s - loss: 0.9026 - acc: 0.7500 - val_loss: 2.1022 - val_acc: 0.3438
Epoch 9/12
64/64 [==============

In [59]:
model.optimizer.lr=0.001

In [60]:
model.fit_generator(batches, batches.batch_size, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/12
64/64 [==============================] - 0s - loss: 0.7213 - acc: 0.7500 - val_loss: 2.1187 - val_acc: 0.1406
Epoch 2/12
64/64 [==============================] - 0s - loss: 0.7099 - acc: 0.7812 - val_loss: 2.1266 - val_acc: 0.1406
Epoch 3/12
64/64 [==============================] - 0s - loss: 0.6063 - acc: 0.8438 - val_loss: 2.2760 - val_acc: 0.0469
Epoch 4/12
64/64 [==============================] - 0s - loss: 0.6367 - acc: 0.7656 - val_loss: 2.1614 - val_acc: 0.1406
Epoch 5/12
64/64 [==============================] - 0s - loss: 0.6733 - acc: 0.7812 - val_loss: 2.0990 - val_acc: 0.1250
Epoch 6/12
64/64 [==============================] - 0s - loss: 0.6695 - acc: 0.8438 - val_loss: 2.1295 - val_acc: 0.1719
Epoch 7/12
64/64 [==============================] - 0s - loss: 0.6608 - acc: 0.7500 - val_loss: 2.2319 - val_acc: 0.1250
Epoch 8/12
64/64 [==============================] - 0s - loss: 0.4448 - acc: 0.8125 - val_loss: 2.3699 - val_acc: 0.1250
Epoch 9/12
64/64 [==============

## Batchnorm + dropout + data augmentation

In [61]:
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [62]:
model = get_model_bn_do()

/home/farlion/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_6 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [63]:
model.fit_generator(batches, batches.batch_size, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/1
64/64 [==============================] - 0s - loss: 3.8087 - acc: 0.1094 - val_loss: 2.3007 - val_acc: 0.0938


In [64]:
model.optimizer.lr=0.1

In [65]:
model.fit_generator(batches, batches.batch_size, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/4
64/64 [==============================] - 0s - loss: 3.2554 - acc: 0.1406 - val_loss: 2.2979 - val_acc: 0.1406
Epoch 2/4
64/64 [==============================] - 0s - loss: 2.3838 - acc: 0.3438 - val_loss: 2.2873 - val_acc: 0.1094
Epoch 3/4
64/64 [==============================] - 0s - loss: 2.7289 - acc: 0.2656 - val_loss: 2.2710 - val_acc: 0.1562
Epoch 4/4
64/64 [==============================] - 0s - loss: 2.1468 - acc: 0.4219 - val_loss: 2.2336 - val_acc: 0.2969


In [66]:
model.optimizer.lr=0.01

In [67]:
model.fit_generator(batches, batches.batch_size, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/12
64/64 [==============================] - 0s - loss: 2.1458 - acc: 0.4219 - val_loss: 2.2252 - val_acc: 0.3125
Epoch 2/12
64/64 [==============================] - 0s - loss: 1.3473 - acc: 0.5781 - val_loss: 2.1898 - val_acc: 0.4844
Epoch 3/12
64/64 [==============================] - 0s - loss: 1.6321 - acc: 0.5156 - val_loss: 2.1968 - val_acc: 0.4531
Epoch 4/12
64/64 [==============================] - 0s - loss: 1.5997 - acc: 0.5312 - val_loss: 2.1828 - val_acc: 0.4531
Epoch 5/12
64/64 [==============================] - 0s - loss: 1.4159 - acc: 0.5781 - val_loss: 2.1551 - val_acc: 0.4219
Epoch 6/12
64/64 [==============================] - 0s - loss: 1.0849 - acc: 0.6406 - val_loss: 2.1225 - val_acc: 0.5625
Epoch 7/12
64/64 [==============================] - 0s - loss: 0.8848 - acc: 0.7188 - val_loss: 2.1349 - val_acc: 0.5312
Epoch 8/12
64/64 [==============================] - 0s - loss: 1.1334 - acc: 0.6562 - val_loss: 2.0976 - val_acc: 0.5156
Epoch 9/12
64/64 [==============

In [68]:
model.optimizer.lr=0.001

In [69]:
model.fit_generator(batches, batches.batch_size, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.batch_size)

Epoch 1/1
64/64 [==============================] - 0s - loss: 0.9446 - acc: 0.6719 - val_loss: 2.0939 - val_acc: 0.3281


## Ensembling

In [70]:
def fit_model():
    model = get_model_bn_do()
    model.fit_generator(batches, batches.batch_size, nb_epoch=1, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.batch_size)
    model.optimizer.lr=0.1
    model.fit_generator(batches, batches.batch_size, nb_epoch=4, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.batch_size)
    model.optimizer.lr=0.01
    model.fit_generator(batches, batches.batch_size, nb_epoch=12, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.batch_size)
    model.optimizer.lr=0.001
    model.fit_generator(batches, batches.batch_size, nb_epoch=18, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.batch_size)
    return model

In [71]:
models = [fit_model() for i in range(6)]

/home/farlion/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_7 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/home/farlion/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_8 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/home/farlion/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_9 and cannot be automatically inferred with the Th

In [72]:
path = "data/mnist/"
model_path = path + 'models/'

In [74]:
for i,m in enumerate(models):
    m.save_weights(model_path+'cnn-mnist23-'+str(i)+'.pkl')

In [75]:
evals = np.array([m.evaluate(X_test, y_test, batch_size=256) for m in models])

 9984/10000 [============================>.] - ETA: 0s

In [76]:
evals.mean(axis=0)

array([ 2.2022,  0.1613])

In [77]:
all_preds = np.stack([m.predict(X_test, batch_size=256) for m in models])

In [78]:
all_preds.shape

(6, 10000, 10)

In [79]:
avg_preds = all_preds.mean(axis=0)

In [80]:
keras.metrics.categorical_accuracy(y_test, avg_preds).eval()

array(0.11580000072717667, dtype=float32)